In [18]:
from enum import Enum
from collections import namedtuple
from typing import Deque
import random

class Cell(Enum):
    EMPTY = ' '
    BLOCKED = 'X'
    START = 'S'
    GOAL = 'G'
    PATH = '*'
    
MazeLocation = namedtuple('MazeLocation', ['row', 'column'])

class Maze:
    def __init__(self, rows=10, columns=10, sparsity=0.2, start=MazeLocation(0,0), goal=MazeLocation(9,9)):
        self.rows = rows
        self.columns = columns
        self.sparsity = sparsity
        self.start = start
        self.goal = goal
        self.grid = [[Cell.EMPTY for _ in range(self.columns)] for _ in range(self.rows)]
        self.random_fill()
        self.grid[self.start.row][self.start.column] = Cell.START
        self.grid[self.goal.row][self.goal.column] = Cell.GOAL
    def random_fill(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if random.uniform(0,1) < self.sparsity:
                    self.grid[row][column] = Cell.BLOCKED
    def __str__(self):
        out = ''
        for row in range(self.rows):
            out += ''.join([c.value for c in self.grid[row]]) + '\n'
        return out
    def goal_test(self, ml):
        return ml == self.goal
    def successor(self, ml):
        locations = []
        if ml.row + 1 < self.rows and self.grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self.grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self.columns and self.grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self.grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    def mark(self, path):
        for ml in path:
            self.grid[ml.row][ml.column] = Cell.PATH
        self.grid[self.start.row][self.start.column] = Cell.START
        self.grid[self.goal.row][self.goal.column] = Cell.GOAL
    def unmark(self, path):
        for ml in path:
            self.grid[ml.row][ml.column] = Cell.EMPTY
        self.grid[self.start.row][self.start.column] = Cell.START
        self.grid[self.goal.row][self.goal.column] = Cell.GOAL
        
class Stack:
    def __init__(self):
        self.collection = []
    @property
    def is_empty(self):
        return not self.collection
    def push(self, item):
        self.collection.append(item)
    def pop(self):
        return self.collection.pop()
    def __repr__(self):
        return repr(self.collection)
    
class Queue:
    def __init__(self):
        self.collection = Deque([])
    @property
    def is_empty(self):
        return not self.collection
    def push(self, item):
        self.collection.append(item)
    def pop(self):
        return self.collection.popleft()
    def __repr__(self):
        return repr(self.collection)
    
class Node:
    def __init__(self, state, parent, cost=0.0, heuristic=0.0):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic
    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)
    
def node2path(node):
    if node is None:
        print('no solution')
        return []
    path = [node.state]
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    return path[::-1]

def dfs(initial, goal_test, successor):
    frontier = Stack()
    frontier.push(Node(initial, None))
    explored = {initial}
    while not frontier.is_empty:
        current_node = frontier.pop()
        current_state = current_node.state
        if goal_test(current_state):
            return current_node
        for child in successor(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None

def bfs(initial, goal_test, successor):
    frontier = Queue()
    frontier.push(Node(initial, None))
    explored = {initial}
    while not frontier.is_empty:
        current_node = frontier.pop()
        current_state = current_node.state
        if goal_test(current_state):
            return current_node
        for child in successor(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None


In [20]:
N = 30
maze = Maze(rows=N, columns=N, goal=MazeLocation(N-1, N-1))
df_path = node2path(dfs(maze.start, maze.goal_test, maze.successor))
maze.mark(df_path)
print(f"depth first search\n{maze}\n")
maze.unmark(df_path)
bf_path = node2path(bfs(maze.start, maze.goal_test, maze.successor))
maze.mark(bf_path)
print(f"breadth first search\n{maze}\n")
maze.unmark(bf_path)

depth first search
S*** XX   X X     XXXX        
X  *XXXX   X         X   X    
 ***                   X    XX
X*X      X   XX   X           
** X XX        X     X    X  X
*XX      XXXX XX      *****X  
**X  XX ***************X  *** 
 ********X          XX    X *X
  X X X     X        X X **** 
X XX        XXX****X  X X* XX 
    XX********** X********    
     X*  X     X     X    X X 
*******         X   X  X     X
*  X   ************XX   ******
*** X  *       X  **X  **X X *
  *X XX**    X  X  *****X  X**
 **     *X X     X     X  X *X
X*X *****X X  X  X   X     X**
 ****X X    XX X X *****    X*
    X  X XXX X ***X* XX*******
 X    X X   X X*X***      X   
  X      X******   X          
XX********* XX       X        
***                  X   X  X 
*            X   X  X  XX     
*X  X  X *****X X   *****X    
*XX*******XX ********XX ******
****X      X X    XX         *
X X   X   XX  X    X    X    *
X                X    X      G


breadth first search
S*   XX   X X     XXXX      